In [ ]:
from re import S
import tensorflow
import pandas
import numpy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras import regularizers
import os
import matplotlib.pyplot as plt


# Dont tweak
IMAGE_SIZE = 512
NUM_CHANNELS = 3

INITIAL_LEARNING_RATE = 0.001
L1_REGULARIZATION = 1e-5
L2_REGULARIZATION = 1e-4
BATCH_SIZE = 32
BINARY_CLASSIFICATION = True


class Dataset:

  def load(self, datasetPath):
    # Load images from directory into tensor
    # Sub-directories specify the class label of the image
    datagenerator = ImageDataGenerator(
        rotation_range=30,
        shear_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest',
        brightness_range=[0.5, 1.5],
        rescale=1.0/255.0   # Normalize
    )

    # Load training data
    self.training_data = datagenerator.flow_from_directory(
        datasetPath + "/train",
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        class_mode='binary',
        shuffle=True,
        batch_size=BATCH_SIZE
    )

    # Load validation data
    self.validation_data = datagenerator.flow_from_directory(
        datasetPath + "/valid",
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        class_mode='binary',
        shuffle=True,
        batch_size=BATCH_SIZE
    )

    validation_data = self.validation_data.next()
    training_data = self.training_data.next()


    # Preserve the number of classes
    self.numClasses = self.training_data.num_classes

class ConvNeuralNet:

  def __init__(self, dataset):
    self.dataset = dataset

  # Add a convolutional layer of the specified parameters to the network
  # PreserveInputSize = Layer will have same number of input nodes as output nodes

  # IO Layers
  def addInputLayer(self, shape):
    self.inputLayer = keras.layers.Input(shape=shape, batch_size=BATCH_SIZE)
    return self.inputLayer

  def addOutputLayer(self, prevLayer):
    if (BINARY_CLASSIFICATION):
      self.outputLayer = keras.layers.Dense(1, activation='sigmoid')(prevLayer)
    else:
      self.outputLayer = keras.layers.Dense(self.dataset.numClasses, activation='softmax')(prevLayer)
    return self.outputLayer

  # General Layers
  def addConvolutionalLayer(self, prevLayer, numFilters, shape, regularize=False):
    kernel_regularizers = None
    if (regularize):
      kernel_regularizers = regularizers.L1L2(l1=L1_REGULARIZATION, l2=L2_REGULARIZATION)
    return keras.layers.Conv2D(numFilters, shape, padding='same', kernel_regularizer=kernel_regularizers)(prevLayer)

  def addDenseLayer(self, prevLayer, units, dropout=0.0, regularize=False):
    kernel_regularizers = None
    if (regularize):
      kernel_regularizers = regularizers.L1L2(l1=L1_REGULARIZATION, l2=L2_REGULARIZATION)
    dense = keras.layers.Dense(units, kernel_regularizer=kernel_regularizers, activation="relu")(prevLayer)
    return keras.layers.Dropout(dropout)(dense)

  def addDropoutLayer(self, prevLayer, rate):
    return keras.layers.Dropout(rate)(prevLayer)

  # Local Pooling
  def addMaxPoolingLayer(self, prevLayer, shape, strides=2):
    return keras.layers.MaxPooling2D(shape, strides=strides)(prevLayer)

  def addAvgPoolingLayer(self, prevLayer, shape, strides=2):
    return keras.layers.AveragePooling2D(shape, strides=strides)(prevLayer)

  # Global Pooling
  def addFlattenLayer(self, prevLayer):
    return keras.layers.Flatten()(prevLayer)

  def addGlobalAveragePoolingLayer(self, prevLayer):
    return keras.layers.GlobalAveragePooling2D()(prevLayer)

  def addGlobalMaxPoolingLayer(self, prevLayer):
    return keras.layers.GlobalMaxPooling2D()(prevLayer)

  def trainAndTest(self, numEpochs):
    optimizer = Adam(learning_rate=INITIAL_LEARNING_RATE)

    reduce_lr = ReduceLROnPlateau(patience=4)
    early_stopping = EarlyStopping(patience=10, monitor="val_loss")
    #model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)
    self.model = keras.models.Model(inputs=[self.inputLayer], outputs=[self.outputLayer])
    if (BINARY_CLASSIFICATION):
      self.model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    else:
      self.model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    self.model.summary()
    history = self.model.fit(
        self.dataset.training_data,
        epochs = numEpochs,
        validation_data = self.dataset.validation_data,
        batch_size = BATCH_SIZE,
        callbacks=[reduce_lr, early_stopping]
    )

    # PLOT ACCURACY VS EPOCH GRAPH
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.plot(history.history['val_accuracy'], label='val_accuracy')
    plt.title('model accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Validation'], loc='lower right')
    plt.show()

    # PLOT LOSS VS EPOCH GRAPH
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.title('model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper right')
    plt.show()

  # Complex Blocks
  def addDenseBlock(self, prevLayer, numFilters, dropout=0.0, regularize=False):
    kernel_regularizers = None
    if (regularize):
      kernel_regularizers = regularizers.L1L2(l1=L1_REGULARIZATION, l2=L2_REGULARIZATION)

    conv1 = keras.layers.BatchNormalization()(prevLayer)
    conv1 = keras.layers.Activation("relu")(conv1)
    conv1 = keras.layers.Conv2D(numFilters, (3, 3), padding='same', kernel_regularizer=kernel_regularizers)(conv1)
    conv1 = keras.layers.Dropout(dropout)(conv1)

    conv2 = keras.layers.BatchNormalization()(conv1)
    conv2 = keras.layers.Activation("relu")(conv2)
    conv2 = keras.layers.Conv2D(numFilters, (3, 3), padding='same', kernel_regularizer=kernel_regularizers)(conv2)
    conv2 = keras.layers.Dropout(dropout)(conv2)
    concat = keras.layers.Concatenate()([conv1, conv2])

    conv3 = keras.layers.BatchNormalization()(concat)
    conv3 = keras.layers.Activation("relu")(conv3)
    conv3 = keras.layers.Conv2D(numFilters, (3, 3), padding='same', kernel_regularizer=kernel_regularizers)(conv3)
    conv3 = keras.layers.Dropout(dropout)(conv3)
    concat = keras.layers.Concatenate()([conv1, conv2, conv3])

    conv4 = keras.layers.BatchNormalization()(concat)
    conv4 = keras.layers.Activation("relu")(conv4)
    conv4 = keras.layers.Conv2D(numFilters, (3, 3), padding='same', kernel_regularizer=kernel_regularizers)(conv4)
    conv4 = keras.layers.Dropout(dropout)(conv4)
    concat = keras.layers.Concatenate()([conv2, conv3, conv4])
    return concat

  def addDownsampleBlock(self, prevLayer, numFilters, dropout=0.0, regularize=False, padding="valid"):
    kernel_regularizers = None
    if (regularize):
      kernel_regularizers = regularizers.L1L2(l1=L1_REGULARIZATION, l2=L2_REGULARIZATION)

    conv = keras.layers.BatchNormalization()(prevLayer)
    conv = keras.layers.Activation("relu")(conv)
    conv = keras.layers.Conv2D(numFilters, (1, 1), padding=padding, kernel_regularizer=kernel_regularizers)(conv)
    conv = keras.layers.Dropout(dropout)(conv)
    conv = keras.layers.MaxPooling2D((2, 2), strides=2)(conv)
    return conv

  def addConcatPooling(self, prevLayer):
    maxPool = self.addMaxPoolingLayer(prevLayer, (2, 2))
    avgPool = self.addAvgPoolingLayer(prevLayer, (2, 2))
    concat = self.addConcat([maxPool, avgPool])
    return concat

# Load dataset
dataset = Dataset()
dataset.load("elbowenh") # path and testing split size

cnn = ConvNeuralNet(dataset) # This will automatically make an input layer for the dataset

# Input layers
input = cnn.addInputLayer((IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))
conv = cnn.addConvolutionalLayer(input, 16, (3, 3))
conv = cnn.addMaxPoolingLayer(conv, (3, 3))

# Dense blocks
block = cnn.addDenseBlock(conv, 32)
block = cnn.addDownsampleBlock(block, 32)
block = cnn.addDenseBlock(block, 64)
block = cnn.addDownsampleBlock(block, 64)
block = cnn.addDenseBlock(block, 128)
block = cnn.addDownsampleBlock(block, 128)
block = cnn.addDenseBlock(block, 256)

# Dense layers
flat = cnn.addGlobalAveragePoolingLayer(block)
dense = cnn.addDenseLayer(flat, 256)
dense = cnn.addDenseLayer(dense, 128)
cnn.addOutputLayer(dense)

# Train and test
cnn.trainAndTest(1000) # 1000 Epochs

Found 4931 images belonging to 2 classes.
Found 465 images belonging to 2 classes.
Model: "model_14"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_29 (InputLayer)       [(32, 512, 512, 3)]          0         []                            
                                                                                                  
 conv2d_275 (Conv2D)         (32, 512, 512, 16)           448       ['input_29[0][0]']            
                                                                                                  
 max_pooling2d_144 (MaxPool  (32, 255, 255, 16)           0         ['conv2d_275[0][0]']          
 ing2D)                                                                                           
                                                                                                  
 batch_n

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Apr 15 18:18:48 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0              65W / 400W |  35553MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
i = []
while True:
  i.append(i)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!7z x drive/MyDrive/elbowenh.zip


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,12 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (50657),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 452774033 bytes (432 MiB)

Extracting archive: drive/MyDrive/elbowenh.zip
 75% 4096 Open              --
Path = drive/MyDrive/elbowenh.zip
Type = zip
Physical Size = 452774033

  0%      7% 407 - elbowenh/train/frac/IMG_elbow_00405.png                                                   13% 719 - elbowenh/train/frac/IMG_elbow_00717.png                                                   18% 990 - elbowenh/train/frac/IMG_elbow_00988.png   